In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, 
                             classification_report)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from google.cloud import aiplatform
from datetime import datetime
import joblib
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
def select_features(X_train, y_train, X_test, feature_names, k=30):
    """
    Select top k features using ANOVA F-value
    
    Parameters:
    X_train: numpy array of scaled training features
    y_train: numpy array of training target
    X_test: numpy array of scaled test features
    feature_names: list of original feature names
    k: number of top features to select
    """
    selector = SelectKBest(score_func=f_classif, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    # Get selected feature names using the original feature names
    selected_features = [feature_names[i] for i in selector.get_support(indices=True)]
    
    return X_train_selected, X_test_selected, selected_features

# Get feature names before scaling (when X_train is still a DataFrame)
feature_names = X_train.columns.tolist()

# Select top 30 features (adjust based on your needs)
X_train_selected, X_test_selected, selected_features = select_features(
    X_train_scaled, y_train_res, X_test_scaled, feature_names, k=30
)

print("Selected Features:")
print(selected_features)

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train_res:", y_train_res.shape)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, roc_auc_score, confusion_matrix,
                            classification_report)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from google.cloud import aiplatform
from datetime import datetime
import joblib
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from sklearn.compose import ColumnTransformer

# **After installing the compatible version, run this again to verify:**
import sklearn
print(f"Scikit-learn version after potential install: {sklearn.__version__}")

# Load the dataset
data = pd.read_excel('US_Recession.xlsx')  # Update with your actual file path

if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

# Preprocess the data
def preprocess_data(df):
    df_processed = df.copy()
    df_processed.fillna(method='ffill', inplace=True)
    df_processed.fillna(method='bfill', inplace=True)
    if 'gdp_growth' in df.columns:
        for lag in [1, 2, 4]:
            df_processed[f'gdp_growth_lag_{lag}'] = df_processed['gdp_growth'].shift(lag)
    if 'unemployment_rate' in df.columns:
        df_processed['unemployment_change'] = df_processed['unemployment_rate'].diff()
    if 'industrial_production' in df.columns:
        df_processed['industrial_prod_rolling_mean_4q'] = (
            df_processed['industrial_production'].rolling(window=4).mean()
        )
    df_processed.dropna(inplace=True)
    return df_processed

processed_data = preprocess_data(data)
X = processed_data.drop('recession_indicator', axis=1)
y = processed_data['recession_indicator']
split_idx = int(0.8 * len(X))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
scaler = StandardScaler()
X_train_scaled_res = scaler.fit_transform(X_train_res) # Scale the resampled training data
X_test_scaled = scaler.transform(X_test)

def select_features(X_train, y_train, X_test, feature_names, k=30):
    selector = SelectKBest(score_func=f_classif, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    selected_features_indices = selector.get_support(indices=True)
    selected_features = [feature_names[i] for i in selected_features_indices]
    return X_train_selected, X_test_selected, selected_features, selected_features_indices

feature_names = X_train.columns.tolist() # Original feature names

# Select features based on the scaled resampled training data and resampled target
X_train_selected_res, X_test_selected, selected_features, selected_features_indices = select_features(
    X_train_scaled_res, y_train_res, X_test_scaled, feature_names, k=30
)

def train_and_evaluate_models(X_train, y_train, X_test, y_test):
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, class_weight='balanced'),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, max_depth=5, random_state=42),
        'XGBoost': XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.05, objective='binary:logistic', random_state=42, scale_pos_weight=sum(y_train==0)/sum(y_train==1))
    }
    best_model = None
    best_score = 0.0
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train) # Use X_train_selected_res here
        y_pred = model.predict(X_test) # Use X_test_selected here
        y_prob = model.predict_proba(X_test)[:, 1] # Use X_test_selected here
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_prob)
        results[name] = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'roc_auc': roc_auc}
        print(f"\n{name} Classification Report:")
        print(classification_report(y_test, y_pred))
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(5, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'{name} Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.show()
        if roc_auc > best_score:
            best_score = roc_auc
            best_model = model
    print("\nModel Comparison:")
    for model_name, metrics in results.items():
        print(f"\n{model_name}:")
        for metric_name, value in metrics.items():
            print(f"{metric_name}: {value:.4f}")
    return best_model

# Train and evaluate models using the selected resampled data
best_model = train_and_evaluate_models(X_train_selected_res, y_train_res, X_test_selected, y_test)

# Create the full pipeline with the *fitted* scaler and feature selector
full_pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        [('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', scaler)]),
          selected_features)],
        remainder='passthrough'
    )),
    ('model', best_model)
])

# **FIT THE PIPELINE TO THE SCALED AND SELECTED RESAMPLED TRAINING DATA:**
full_pipeline.fit(X_train_res.iloc[:, selected_features_indices], y_train_res)

# Save the *entire* fitted pipeline
joblib.dump(full_pipeline, 'vertex_model.joblib')
print("✅ Full pipeline saved as vertex_model.joblib")

# Upload the pipeline to GCS
from google.cloud import storage
client = storage.Client.from_service_account_json(
    r"C:/Users/rudra.s/Downloads/spry-scope-456205-m7-be989ff0ac36.json"
)
blob = client.bucket('us-recession-model').blob('vertex_model.joblib')
blob.upload_from_filename('vertex_model.joblib', content_type='application/joblib')
print("✅ Pipeline uploaded to gs://us-recession-model/vertex_model.joblib")

In [ ]:
pip install scikit-learn==1.5.2

In [ ]:
import sys
print(sys.executable)